# Chlorophyll Project - NAN Method
#### There are seventy seabird observational sites across the globe. We want to gather chlorophyll data for a 1km diameter around these points, in a circle.
#### This method will use a distance function to only render points within a 300km diameter range around any seabird site.

In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
import geopy.distance

#load map packages
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
import cartopy.crs as ccrs

In [ ]:
# make a function (called below) that makes a mask (2d dataset where values to retain are set to 1 and others to nan)
# for a 300km radius area around a given lat/lon pair. 
# note that the area in km is not the same as area in degrees. longitude degrees change distance with latitude.

In [ ]:
# if acquiring for the first time (1)
ds_color = xr.open_dataset('https://rsg.pml.ac.uk/thredds/dodsC/CCI_ALL-v5.0-MONTHLY')

In [ ]:
# open CSV
locs = pd.read_csv('../NSF_Seabirdsloc_Summer2022.csv')
sites = locs['Site']

# random site
site = sites[np.random.randint(0,len(sites))]
# if (site == ""):
#     site = 'Adams'
sites = sites.tolist()

lon = locs.loc[[sites.index(site)]]['Longitude']
lat = locs.loc[[sites.index(site)]]['Latitude']

print("Working site " + site)

In [ ]:
# open CSV

# copy to prevent overwriting
def findSiteAvgLevel(site, plot=False):
    data = ds_color.copy()

    # for each location, select a region +-5 deg, all times
    lats = [float(lat-5), float(lat+5)]
    lons = [float(lon-5), float(lon+5)]

    data = data['chlor_a'][291].sel(
        lon = slice(lons[0], lons[1]),
        lat = slice(lats[1], lats[0])
    ).load()

    # DEBUG: print lats and lons range
    # print("Mapping between " + str(lats[0]) + " and " + str(lats[1]))
    # print("Mapping between " + str(lons[0]) + " and " + str(lons[1]))

    # get the distance between two points of lat,lon in km (from point 0 on equator)
    def fromEquatorDistance(lat1, lon1, lat2, lon2):
        p = 0.017453292519943295 # Pi/180
        a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
        d = 12742 * np.arcsin(np.sqrt(a)) # 2*R*asin...
        return d

    latPos = lat.values.item()
    lonPos = lon.values.item()
    # plot values only that are within 300 units of the (lat, lon)
    sumChLevels = 0;
    totalPointsInRange = 0;
    for i in range(len(data.lat)):
        for j in range(len(data.lon)):
            thisLat = data.lat[i].item()
            thisLon = data.lon[j].item()

            # This method uses the great circle method, using a spherical model of the earth to calculate distance.
            # It is 20x faster than geopy.distance.distance(), but is notably less accurate,
            # with a 0.5% error.
            # While most likely unnecessary, this method is included for completeness.
            # if (geopy.distance.distance((thisLat, thisLon), (latPos, lonPos)).km >= 300):
            #    data.values[i][j] = np.nan

            if (geopy.distance.great_circle((thisLat, thisLon), (latPos, lonPos)).km >= 300):
               data.values[i][j] = np.nan
            else:
                pt = data.values[i][j]
                #nan
                if (str(pt) == "nan"):
                    continue;
                sumChLevels += data.values[i][j].item()
                totalPointsInRange += 1

    avgChLevel = sumChLevels/totalPointsInRange

    region = np.array([
        [
            data.lat.min().values.item(),
            data.lat.max().values.item()
        ],
        [
            data.lon.min().values.item(),
            data.lon.max().values.item()
        ]
    ]) 

    # # If you want a graph for the site, uncomment below:
    if (plot == True):
    # # Select a region of our data
        fig = plt.figure(figsize=(16,10), dpi = 72)
        ax = plt.axes(projection=ccrs.PlateCarree())



        states_provinces = cfeature.NaturalEarthFeature(
            category='cultural',
            name='admin_1_states_provinces_lines',
            scale='50m',
            facecolor='none')
        ax.coastlines(resolution='10m',linewidth=1,color='black')
        ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
        ax.add_feature(states_provinces, linewidth = 0.5)
        ax.add_feature(cfeature.BORDERS, color = 'black')
        ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
        ax.yaxis.set_major_formatter(LatitudeFormatter())
        ax.gridlines(linestyle = '--', linewidth = 0.5)    #plot diff data
        plt.title("Seabird Observation Locations")
        ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree())
        data.plot()
        plt.show()

    return avgChLevel

    # pick a random point in the data, and calculate its distance to the site.

    # take a mean in space, and store the time series into a numpy array

    # save the arrays as dataarray in a dataset

    # add locations as other variables 

In [ ]:
# Example Usage: Find the average chlorophyll level for a specific site
# (and plot the data if you want)
findSiteAvgInLevel(sites.index("Adams"), plot=True)
# Example Usage: Find the average chlorophyll level for every site
for x in range(len(sites)):
    chl = findSiteAvgLevel(sites[x])
    print(sites[x] + ": " + str(chl))